In [ ]:
import os
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence

In [ ]:
def create_unet_model(input_shape, n_classes):
    img_input = Input(shape=input_shape)

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

    up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

    out = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(conv5)

    model = Model(inputs=img_input, outputs=out)
    return model

In [ ]:
# Custom Data Generator for Segmentation
class SegmentationDataGenerator(Sequence):
    def __init__(self, image_dir, mask_dir, batch_size=32, image_size=(256, 256)):
        self.image_paths = [os.path.join(image_dir, fname) for fname in sorted(os.listdir(image_dir)) if fname.endswith('.jpg')]
        self.mask_paths = [os.path.join(mask_dir, fname) for fname in sorted(os.listdir(mask_dir)) if fname.endswith('.png')]
        self.batch_size = batch_size
        self.image_size = image_size
        self.indexes = np.arange(len(self.image_paths))

        # Check that number of images matches number of masks
        assert len(self.image_paths) == len(self.mask_paths), "Mismatch in number of images and masks"

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images = [self.load_image(self.image_paths[i]) for i in batch_indexes]
        batch_masks = [self.load_mask(self.mask_paths[i]) for i in batch_indexes]
        
        # Debugging: Print batch shapes
#         print(f"Batch images shape: {np.array(batch_images).shape}")
#         print(f"Batch masks shape: {np.array(batch_masks).shape}")

        return np.array(batch_images), np.array(batch_masks)

    def load_image(self, path):
        img = load_img(path, target_size=self.image_size)
        img_array = img_to_array(img)
        return img_array / 255.0  # Normalize

    def load_mask(self, path):
        mask = load_img(path, target_size=self.image_size, color_mode='grayscale')  # Load mask as grayscale
        mask_array = img_to_array(mask).astype(np.uint8)

        # Convert mask values to integers (assuming 38 is the only non-zero value)
        mask_array = np.where(mask_array == 38, 1, 0)  # Map pixel value 38 to 1, all else to 0

        return mask_array

In [ ]:
# Define your directories here
image_dir = '/kaggle/input/leaf-disease-segmentation-dataset/aug_data/aug_data/images'  # .jpg images
mask_dir = '/kaggle/input/leaf-disease-segmentation-dataset/aug_data/aug_data/masks'  # .png masks

In [ ]:
# Create data generator
train_gen = SegmentationDataGenerator(
    image_dir=image_dir,
    mask_dir=mask_dir,
    batch_size=16,
    image_size=(256, 256)
)

In [ ]:
# Create the U-Net model
model = create_unet_model(input_shape=(256, 256, 3), n_classes=2)

In [ ]:
from tensorflow.keras.optimizers import Adam
# Example usage
def train_model(model, train_gen, epochs):
    optimizer = Adam(learning_rate=1e-4)  # Lower the learning rate
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        train_gen,
        epochs=epochs,
        steps_per_epoch=len(train_gen)
    )

In [ ]:
# Train the model
train_model(
    model,
    train_gen,
    epochs=500
)

In [ ]:
model.save('unet_500.keras')